In [1]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

print("Modern LangChain v1+ - Union Budget RAG ready!")

print("Union Budget 2025-26 RAG Analysis")
print("Loading ALL PDFs")

Modern LangChain v1+ - Union Budget RAG ready!
Union Budget 2025-26 RAG Analysis
Loading ALL PDFs


In [2]:
from langchain_community.document_loaders import PyPDFLoader

# Load just 3 key budget files to understand
pdf_dir = "../data/pdfs/"
key_files = ['Budget_Speech.pdf', 'budget_at_a_glance.pdf', 'allafs.pdf']

docs = []
for pdf_file in key_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    if os.path.exists(pdf_path):
        loader = PyPDFLoader(pdf_path)
        file_docs = loader.load()
        docs.extend(file_docs)
        print(f"{pdf_file}: {len(file_docs)} pages loaded")
    else:
        print(f"{pdf_file} not found")

print(f"\nTOTAL: {len(docs)} documents loaded")
print(f"First doc source: {os.path.basename(docs[0].metadata['source']) if docs else 'None'}")
print(f"First doc preview: {docs[0].page_content[:200]}...")

Budget_Speech.pdf: 60 pages loaded
budget_at_a_glance.pdf: 23 pages loaded
allafs.pdf: 18 pages loaded

TOTAL: 101 documents loaded
First doc source: Budget_Speech.pdf
First doc preview: GOVERNMENT OF INDIA
BUDGET 2025-2026
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2025...


In [3]:
# breakdown by document

sources = {}
for doc in docs:
    source = os.path.basename(doc.metadata['source'])
    sources[source] = sources.get(source, 0) + 1

print("Pages by document:")
for source, count in sources.items():
    print(f"  {source}: {count} pages")

print(f"\nFirst page length: {len(docs[0].page_content)} characters")
print(f"Sample from page 2:")
print(docs[1].page_content[:200] + "...")

Pages by document:
  Budget_Speech.pdf: 60 pages
  budget_at_a_glance.pdf: 23 pages
  allafs.pdf: 18 pages

First page length: 103 characters
Sample from page 2:
...


In [4]:
# Analyze what we loaded

sources = {}
page_lengths = []
for doc in docs:
    source = os.path.basename(doc.metadata['source'])
    sources[source] = sources.get(source, 0) + 1
    page_lengths.append(len(doc.page_content))

print("Pages by document:")
for source, count in sources.items():
    print(f"  {source}: {count} pages")

print(f"\nStats:")
print(f"  Total pages: {len(docs)}")
print(f"  Avg page length: {sum(page_lengths)/len(page_lengths):.0f} chars")
print(f"  Longest page: {max(page_lengths)} chars")

print(f"\nPage 2 sample (Budget Speech):")
print(docs[1].page_content[:300] + "...")

print(f"\nMetadata example:")
print(f"  Source: {docs[0].metadata['source']}")
print(f"  Keys: {list(docs[0].metadata.keys())}")

Pages by document:
  Budget_Speech.pdf: 60 pages
  budget_at_a_glance.pdf: 23 pages
  allafs.pdf: 18 pages

Stats:
  Total pages: 101
  Avg page length: 1959 chars
  Longest page: 5388 chars

Page 2 sample (Budget Speech):
...

Metadata example:
  Source: ../data/pdfs/Budget_Speech.pdf
  Keys: ['producer', 'creator', 'creationdate', 'author', 'moddate', 'title', 'source', 'total_pages', 'page', 'page_label']


### **Splitting the Text into Chunks**

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("BEFORE splitting: 101 long pages")
print(f"Average page: {sum(len(d.page_content) for d in docs)//len(docs):,} chars")
print(f"Longest page: {max(len(d.page_content) for d in docs):,} chars")

# Create the splitter (tries to keep paragraphs together)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # Max 1500 chars per chunk
    chunk_overlap=200,      # 200 char overlap (context preservation)
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Try: paragraphs → sentences → words
)

# SPLIT!
chunks = splitter.split_documents(docs)

print(f"\nAFTER splitting: {len(chunks)} small chunks")
print(f" Average chunk: {sum(len(c.page_content) for c in chunks)//len(chunks):,} chars")
print(f" Max chunk: {max(len(c.page_content) for c in chunks):,} chars")

print(f"\nFIRST CHUNK PREVIEW:")
print(chunks[0].page_content[:400] + "...")
print(f" From: {os.path.basename(chunks[0].metadata['source'])} (page {chunks[0].metadata.get('page', 'N/A')})")

BEFORE splitting: 101 long pages
Average page: 1,959 chars
Longest page: 5,388 chars

AFTER splitting: 191 small chunks
 Average chunk: 1,121 chars
 Max chunk: 1,500 chars

FIRST CHUNK PREVIEW:
GOVERNMENT OF INDIA
BUDGET 2025-2026
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2025...
 From: Budget_Speech.pdf (page 0)


In [6]:
# Enhancing chunks with sources
print("ENHANCING CHUNKS WITH SOURCE TRACKING...")

for chunk in chunks:
    source_file = os.path.basename(chunk.metadata['source'])
    page_num = chunk.metadata.get('page', 1)
    chunk.metadata['file'] = source_file
    chunk.metadata['page'] = page_num

print(f"{len(chunks)} SOURCE-TRACKED chunks ready!")
print(f"Sample → File: {chunks[10].metadata['file']}, Page: {chunks[10].metadata['page']}")

ENHANCING CHUNKS WITH SOURCE TRACKING...
191 SOURCE-TRACKED chunks ready!
Sample → File: Budget_Speech.pdf, Page: 9


### **Generating Embeddings**

Model - *nomic-embed-text*

Context limit - 2k

nomic-embed-text is a large context length text encoder that surpasses OpenAI text-embedding-ada-002 and text-embedding-3-small performance on short and long context tasks.

***"nomic-embed text"* Embedding Model Details**

| Model | Size | Context | Age |
|-------|------|---------|-----|
| **nomic-embed-text:latest** | **274MB** | **2K** | **1 year ago** |

Source: *https://ollama.com/library/nomic-embed-text*

In [7]:
print(" Creating VECTOR DATABASE from chunks...")
print(f" {len(chunks)} chunks → numerical embeddings")

# Create embeddings using Ollama (local!)
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Save to your db/ folder
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="../db/union_budget_vectors"  # From notebooks/ → ../db/
)

print(" VECTOR DATABASE SAVED!")
print(f" Location: ../db/union_budget_vectors/")
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})  # Top 4 relevant chunks
print(" Retriever ready - finds most relevant budget chunks!")

 Creating VECTOR DATABASE from chunks...
 191 chunks → numerical embeddings
 VECTOR DATABASE SAVED!
 Location: ../db/union_budget_vectors/
 Retriever ready - finds most relevant budget chunks!


### **Testing with Llama2 Model**

Llama2 - 4k Context Limit

```
TOTAL BUCKET = 4096 tokens (input + output combined

llama2 TOTAL CAPACITY = 4096 tokens (input + output combined)

RAG INPUT consumes:
├── Question:           ~50 tokens    ("What is fiscal deficit?")
├── Prompt:            ~200 tokens    ("Use this context...")
└── 4 retrieved chunks: 1500 tokens  (375 tokens × k=4)
                             
TOTAL INPUT = ~1750 tokens

RAG Input:     1750 tokens (question + 4 chunks + prompt)
Available Output: 2346 tokens MAX  
TOTAL:         4096 tokens ✓
```

In [8]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Your llama2 model for answering
llm = OllamaLLM(model="llama2")

# RAG prompt template
template = """You are analyzing India's Union Budget 2025-26. 
Use ONLY the following budget document context to answer.

Context from Budget documents:
{context}

Question: {question}

Answer:"""
prompt = ChatPromptTemplate.from_template(template)

# Build RAG chain (Modern LCEL)
rag_chain = (
    {"context": retriever, "question": lambda x: x}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG CHAIN BUILT!")
print("Ready to answer Union Budget questions!")
print("\n Examples to try:")
print('  "What is the fiscal deficit target?"')
print('  "What are the 4 engines of development?"') 
print('  "Total expenditure estimate?"')

RAG CHAIN BUILT!
Ready to answer Union Budget questions!

 Examples to try:
  "What is the fiscal deficit target?"
  "What are the 4 engines of development?"
  "Total expenditure estimate?"


In [9]:
# Testing Our RAG Model with various questions

questions = [
    # Q1-5: SIMPLE (baseline test)
    "What is the fiscal deficit target for 2025-26?",
    "What are the 4 engines of development?",
    "Total expenditure estimate?",
    "What is the capex allocation?",
    "Tax changes for salaried class?",
    
    # Q6-10: MEDIUM (context understanding)
    "Compare fiscal deficit 2024-25 vs 2025-26?",
    "Key agriculture schemes announced?",
    "MSME credit guarantee details?",
    "Export promotion initiatives?",
    "Infrastructure development plan?",
    
    # Q11-15: COMPLEX (multi-hop reasoning)
    "How does 4.4% fiscal target achieved?",
    "Impact of capex on GDP growth?",
    "Tax relief vs revenue measures?",
    "Employment generation strategy?",
    "Budget consolidation roadmap?"
]

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
import time

# ENHANCED PROMPTS
prompt_4k = ChatPromptTemplate.from_template("""
UNION BUDGET 2025-26 ANALYST (4K CONTEXT LIMIT)

CONTEXT (USE ONLY THIS):
{context}

QUESTION: {question}

BRIEF ANSWER (max 2 sentences):
""")

prompt_32k = ChatPromptTemplate.from_template("""
UNION BUDGET 2025-26 FINANCIAL ANALYST (32K CONTEXT)

CONTEXT (Full budget documents):
{context}

DETAILED ANALYSIS REQUIRED:
{question}

COMPREHENSIVE RESPONSE with:
• Key figures/numbers  
• Policy rationale
• Cross-references (tables/speech)
• Implementation details
""")

# MODEL → PROMPT MAPPING
model_prompts = {
    "llama2:latest": prompt_4k,      # 4K = CONCISE
    "gemma3:270m": prompt_32k,       # 32K = DETAILED
    "gemma3:1b": prompt_32k,         # 32K = DETAILED  
    "qwen2:1.5b": prompt_32k,        # 32K = DETAILED
    "qwen:1.8b": prompt_32k,         # 32K = DETAILED
    "qwen2.5:0.5b": prompt_32k,      # 32K = DETAILED
    "qwen:0.5b": prompt_32k          # 32K = DETAILED
}

print(" ENHANCED DUAL PROMPTS LOADED!")
print("  4K models → Brief (2 sentences max)")
print("  32K models → Detailed analysis")

 ENHANCED DUAL PROMPTS LOADED!
  4K models → Brief (2 sentences max)
  32K models → Detailed analysis


In [16]:
# YOUR 7-MODEL BATTLE - METADATA PERFECT!
import time
import pandas as pd

results = []

for model_name, model_prompt in model_prompts.items():
    print(f"\n{model_name.upper()}")
    print("=" * 60)
    
    llm = OllamaLLM(model=model_name, temperature=0.1)
    chain = (
        {"context": retriever, "question": lambda x: x}
        | model_prompt
        | llm
        | StrOutputParser()
    )
    
    model_times = []
    
    for i, question in enumerate(questions):
        start_time = time.time()
        
        # SOURCE TRACKING (WORKS NOW!)
        relevant_docs = retriever.invoke(question)
        sources = [(doc.metadata['file'], doc.metadata['page']+1) for doc in relevant_docs]  # +1 for human-readable
        
        answer = chain.invoke(question)
        elapsed = time.time() - start_time
        
        print(f"Q{i+1:2d} | {elapsed:.1f}s | {question[:35]}...")
        print(f"   SOURCES: {sources}")
        print(f"   {answer[:100]}...")
        print()
        
        model_times.append(elapsed)
    
    avg_time = sum(model_times) / len(model_times)
    context_limit = "4K" if "llama2" in model_name else "32K"
    
    results.append({
        'Model': model_name,
        'Avg_Time': f"{avg_time:.1f}s",
        'Context': context_limit,
        'Prompt': 'Concise' if context_limit == '4K' else 'Detailed'
    })
    
    print(f"{model_name} COMPLETE | AVG: {avg_time:.1f}s")

# FINAL TABLE
print("\n" + "="*80)
print("15-QUESTION CHAMPION TABLE")
print("="*80)
print(pd.DataFrame(results).to_string(index=False))


🏁 LLAMA2:LATEST | CONTEXT BATTLE
Q 1 | 9.7s | What is the fiscal deficit target f...
   📄 SOURCES: [('Budget_Speech.pdf', 23), ('Budget_Speech.pdf', 5), ('Budget_Speech.pdf', 22), ('Budget_Speech.pdf', 1)]
   💬 The fiscal deficit target for 2025-26 is 4.4% of GDP, as estimated in the Budget Speech. This repres...

Q 2 | 12.1s | What are the 4 engines of developme...
   📄 SOURCES: [('Budget_Speech.pdf', 3), ('Budget_Speech.pdf', 6), ('Budget_Speech.pdf', 12), ('Budget_Speech.pdf', 19)]
   💬 The four engines of development in the Union Budget 2025-26 are Agriculture, MSMEs, Investment, and ...

Q 3 | 9.9s | Total expenditure estimate?...
   📄 SOURCES: [('allafs.pdf', 18), ('budget_at_a_glance.pdf', 1), ('Budget_Speech.pdf', 23), ('Budget_Speech.pdf', 22)]
   💬 The total expenditure estimate for Union Budget 2025-26 is ₹50.65 lakh crore. This includes grants f...

Q 4 | 10.6s | What is the capex allocation?...
   📄 SOURCES: [('budget_at_a_glance.pdf', 11), ('budget_at_a_glance.pdf', 10),

| Model         | Avg_Time | Context | Prompt   | Age          |
| ------------- | -------- | ------- | -------- | ------------ | 
| llama2:latest | 11.3s    | 4K      | Concise  | 1 Year Ago   |
| gemma3:270m   | 56.4s    | 32K     | Detailed | 5 months ago |
| gemma3:1b     | 35.8s    | 32K     | Detailed | 9 months ago |
| qwen2:1.5b    | 22.7s    | 32K     | Detailed | 1 year ago   |
| qwen:1.8b     | 1.6s     | 32K     | Detailed | 1 Year Ago   |
| qwen2.5:0.5b  | 56.7s    | 32K     | Detailed | 1 year ago   |
| qwen:0.5b     | 2.0s     | 32K     | Detailed | 1 Year Ago   |